# Modulus that trains a CIFAR 10 dataset
## Uses Resnet50
### Deep Residual Learning for Image Recognition
**Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun**
Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers---8x deeper than VGG nets but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers.
The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.

In [1]:
#!/usr/bin/env python3
"""
Transfer Learning.
"""
import tensorflow.keras as K
import numpy as np


def preprocess_data(X, Y):
    """Preprocess data for Resnet50.
    Args:
        X (np.ndarray): matrix of shape (m, 32, 32, 3) containing the CIFAR 10
                        data, where m is the number of data points.
        Y (np.ndarray): matrix of shape (m,) containing the CIFAR 10 labels
                        for X.
    Returns:
        X is a numpy.ndarray containing the preprocessed X
        Y is a numpy.ndarray containing the preprocessed Y
    """
    X_p = K.applications.resnet50.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p


if __name__ == '__main__':
    (xtn, ytn), (xtt, ytt) = K.datasets.cifar10.load_data()

    xtn, ytn = preprocess_data(xtn, ytn)
    xtt, ytt = preprocess_data(xtt, ytt)

    model = K.applications.ResNet50(include_top=False,
                                    weights='imagenet',
                                    input_shape=(224, 224, 3))
    
    for layer in model.layers[:143]:
      layer.trainable = False

    model_1 = K.Sequential()
    model_1.add(K.layers.UpSampling2D((7, 7)))
    model_1.add(model)
    model_1.add(K.layers.AveragePooling2D(pool_size=7))
    model_1.add(K.layers.Flatten())
    model_1.add(K.layers.Dense(10, activation=('softmax')))

    check = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
                                        monitor='val_acc',
                                        mode='max',
                                        verbose=1,
                                        save_best_only=True)

    model_1.compile(optimizer=K.optimizers.RMSprop(learning_rate=1e-4),
                    loss='categorical_crossentropy',
                    metrics=['acc'])

    model_1.fit(xtn, ytn,
                validation_data=(xtt, ytt),
                batch_size=32,
                epochs=5,
                verbose=1,
                callbacks=[check])

    model_1.save('cifar10.h5')

D:\Programs\anaconda3\envs\tf-gpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/5
1563/1563 [==============================] - 548s 327ms/step - loss: 0.4517 - acc: 0.8493 - val_loss: 0.3983 - val_acc: 0.8825

Epoch 00001: val_acc improved from -inf to 0.88250, saving model to cifar10.h5


D:\Programs\anaconda3\envs\tf-gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/5
1563/1563 [==============================] - 507s 324ms/step - loss: 0.1944 - acc: 0.9343 - val_loss: 0.4775 - val_acc: 0.8829

Epoch 00002: val_acc improved from 0.88250 to 0.88290, saving model to cifar10.h5
Epoch 3/5
1563/1563 [==============================] - 508s 325ms/step - loss: 0.1050 - acc: 0.9641 - val_loss: 0.4148 - val_acc: 0.9040

Epoch 00003: val_acc improved from 0.88290 to 0.90400, saving model to cifar10.h5
Epoch 4/5
1563/1563 [==============================] - 507s 324ms/step - loss: 0.0663 - acc: 0.9791 - val_loss: 0.3548 - val_acc: 0.9102

Epoch 00004: val_acc improved from 0.90400 to 0.91020, saving model to cifar10.h5
Epoch 5/5
1563/1563 [==============================] - 507s 324ms/step - loss: 0.0497 - acc: 0.9840 - val_loss: 0.4684 - val_acc: 0.9104

Epoch 00005: val_acc improved from 0.91020 to 0.91040, saving model to cifar10.h5
